In [5]:
import dask.dataframe as dd
from datetime import datetime
import math
from dask.multiprocessing import get
from dask.distributed import Client, LocalCluster 
print('test')
import dask

#cluster = LocalCluster(n_workers = 4, threads_per_worker = 4, processes=False, memory_limit='4GB', silence_logs='error')
client = Client(processes=False, silence_logs='error')
#client = Client(cluster)
#cluster
client
import timeit

test


In [6]:
# fillna - used because null/None is float, not str, had to make it a str, for max/min
def get_inventory(file_type:str) -> dd:
    if file_type == 'parquet':
        df = dd.read_parquet('./data/checkouts/inventory_parquet', engine="pyarrow")
    elif 'csv' in file_type:
        df = (dd
          .read_csv("./data/checkouts/Library_Collection_Inventory.csv",
                    header=0, 
                    dtype={"BibNum":"int64", "Title":"str","Author":"str", "Subjects":"str"})
         )  
    df = df.drop_duplicates(subset='BibNum')
    return df

def get_inventory_no_dedup(file_type:str) -> dd:
    if file_type == 'parquet':
        df = dd.read_parquet('./data/checkouts/inventory_parquet', engine="pyarrow")
    elif 'csv' in file_type:
        df = (dd
          .read_csv("./data/checkouts/Library_Collection_Inventory.csv",
                    header=0, 
                    dtype={"BibNum":"int64", "Title":"str","Author":"str", "Subjects":"str"})
         )  
    return df
    

def get_checkouts(file_type:str) -> dd:
    if file_type == 'multi-csv':
        df = dd.read_csv("./data/checkouts/Checkouts_By_Title_Data_Lens_*.csv", 
                         header=0, 
                         dtype={"BibNumber":'int64',
                                "ItemBarcode":"str",
                                "ItemType":"str",
                                "Collection":"str",
                                "CallNumber":"str",
                                "CheckoutDateTime":"str"}
                        )
    elif file_type =='single-csv':
        df = dd.read_csv("./data/checkouts/Checkouts.csv", 
                         header=0, 
                         dtype={"BibNumber":'int64',
                                "ItemBarcode":"str",
                                "ItemType":"str",
                                "Collection":"str",
                                "CallNumber":"str",
                                "CheckoutDateTime":"str"}
                        )
    elif file_type =='parquet':
        df = dd.read_parquet('./data/checkouts/checkouts_parquet', engine="pyarrow")  
    return df

def writter_csv(df: dd, scheduler_type, single_file: bool) -> None:
    if single_file:
        df.to_csv('./output/output.csv',single_file=single_file, compute_kwargs={"scheduler": scheduler_type}, mode="w+")
    else:
        df.to_csv('./output/output',single_file=single_file, compute_kwargs={"scheduler": scheduler_type}, mode="w+")
        
def writter_parquet(df: dd, scheduler, flag=None) ->None:
    df.to_parquet("./output/parquet_output")
    
def add_formatted_checkout(file_type:str)->dd:
    df = get_checkouts(file_type)
    df['CheckoutTime_formated']=dd.to_datetime(df.CheckoutDateTime, format="%m/%d/%Y %I:%M:%S %p")
    return df
def add_formatted_checkout_map_partitions(file_type:str)->dd:
    df = get_checkouts(file_type)
    df['CheckoutTime_formated']=df.CheckoutDateTime.map_partitions(pd.to_datetime, 
                                               format="%m/%d/%Y %I:%M:%S %p"
                                                           )
    return df

In [7]:
computes = ["threads", "processes","synchronous"]
file_types = ["parquet", "multi-csv", "single-csv"]
wr = [[writter_csv, False], [writter_parquet,False],[writter_csv, True]]

Reading files

In [8]:
%timeit -n 1 -r 5 get_checkouts("parquet")
%timeit -n 1 -r 5 get_checkouts("multi-csv")
%timeit -n 1 -r 5 get_checkouts("single-csv")

56.9 ms ± 5.87 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
77.9 ms ± 4.63 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
84.2 ms ± 1.54 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


Getting head of read file

In [11]:
%timeit -n 1 -r 5 get_checkouts("parquet").head()
%timeit -n 1 -r 5 get_checkouts("multi-csv").head()
%timeit -n 1 -r 5 get_checkouts("single-csv").head()

2.19 s ± 39.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
1.75 s ± 173 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
1.6 s ± 63.3 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


Reading small file and writting to file

In [ ]:
for compute in computes:
    for file_type in file_types:
        for writter in wr:
            print(f"read + write. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 5 writter[0](get_inventory_no_filter(file_type), compute, writter[1])
            print(f"Read, dedup, write. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 5 writter[0](get_inventory(file_type), compute, writter[1])
            print(f"Checkouts Read + Write. compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 5 writter[0](get_checkouts(file_type), compute, writter[1])
            print(f"Add timestamp and write: compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 5 writter[0](add_formatted_checkout(file_type),compute, writter[1])
            print(f"add timestamp with map partitions + write: compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 5 writter[0](add_formatted_checkout_map_partitions(file_type),compute, writter[1])
            print(f"Get row count: compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 5 get_checkouts(file_type).count().compute()
            print(f"add timestamp, get first 10: compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 5 add_formatted_checkout(file_type).nsmallest(10, "CheckoutTime_formated").head(10)

Read convert to datetime, get min datetime value

In [ ]:
for compute in computes:
    for file_type in file_types:
        stop_list = []
        for i in range(5):
            start = timeit.default_timer()
            df = add_formatted_checkout(file_type)
            df = df.CheckoutTime_formated.min()
            df = dask.optimize(df)[0]
            print(df.compute(scheduler=compute))
            stop = timeit.default_timer()
            stop_list = stop_list.append(stop-start)
        print(f"compute: {compute}, file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

In [ ]:
for compute in computes:
    for file_type in file_types:
        stop_list = []
        for i in range(5):
            start = timeit.default_timer()
            df = add_formatted_checkout_map_partitions(file_type)
            df = df.CheckoutTime_formated.min()
            df = dask.optimize(df)[0]
            print(df.compute(scheduler=compute))
            stop = timeit.default_timer()
            stop_list = stop_list.append(stop-start)
        print(f"compute: {compute}, file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

Inner Join and head

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory("parquet")[["BibNum", "Author"]]
        merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner")
        merged.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

Anti join and head

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory("parquet")[["BibNum", "Author"]]
        merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="outer", indicator=True)
        merged[merged._merge=="right_only"].head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

Read + explode + write

In [ ]:
for compute in computes:
    for file_type in file_types:
        for writter in wr:
            stop_list = []
            for i in range(5):
                start = timeit.default_timer()
                inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
                inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
                inventory = inventory.explode("Subject")
                inventory = dask.optimize(inventory)[0]
                writter[0](inventory, compute, writter[1])
                stop = timeit.default_timer()
                stop_list = stop_list.append(stop-start)
            print(f"compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}, runtime: {sum(stop_list)/len(stop_list)}")

Read+explode+dcount


In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        inventory = inventory.map_partitions(lambda x: x.groupby(x.Subject)["BibNum"].nunique().astype("int64"))
        inventory = dask.optimize(inventory)[0]
        inventory.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"compute: {compute}, file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        inventory = inventory.groupby(inventory.Subject)["BibNum"].nunique().astype("int64"))
        inventory = dask.optimize(inventory)[0]
        inventory.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"compute: {compute}, file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

Read + explode + join + dcount + head

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner")
        merged = merged.map_partitions(lambda x: x.groupby(x.Subject)["BibNum"].nunique().astype("int64"))
        merged = dask.optimize(merged)[0]
        merged.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}, runtime: {sum(stop_list)/len(stop_list)}")

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner")
        merged = merged.groupby(merged.Subject)["BibNum"].nunique().astype("int64")
        merged = dask.optimize(merged)[0]
        merged.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"compute: {compute}, file_type: {file_type}, writter: {writter[0].__name__}, single_file = {writter[1]}, runtime: {sum(stop_list)/len(stop_list)}")